In [44]:
from huggingface_hub import login
import pandas as pd

from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

import time

import warnings
warnings.filterwarnings("ignore")

huggingfacehub_api_token = 'add_your_api_token'

all_results = []

# Import dataset

In [45]:
df = pd.read_csv('../datasets/B2W-Reviews01.csv')
df = df[['product_name']]
df = df.drop_duplicates(subset = "product_name")

/tmp/ipykernel_36/258909019.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../datasets/B2W-Reviews01.csv')


# LLMs

In [47]:
llm_vlt5 = HuggingFaceHub(repo_id='Voicelab/vlt5-base-keywords', 
                          huggingfacehub_api_token=huggingfacehub_api_token,
                          model_kwargs={"max_length":500})

llm_google = HuggingFaceHub(repo_id='google/flan-t5-xxl', 
                            huggingfacehub_api_token=huggingfacehub_api_token,
                            model_kwargs={"max_length":500})

llm_bigscience = HuggingFaceEndpoint(repo_id='bigscience/bloom', 
                                     huggingfacehub_api_token=huggingfacehub_api_token,
                                     model_kwargs={"max_length":500})

llm_face = HuggingFaceHub(repo_id='facebook/blenderbot-400M-distill', 
                          huggingfacehub_api_token=huggingfacehub_api_token,
                          model_kwargs={"max_length":500, 'truncation': 'only_first'})

llm_bode = HuggingFaceHub(repo_id='recogna-nlp/Phi-Bode', 
                               huggingfacehub_api_token=huggingfacehub_api_token,
                               model_kwargs={"max_length":500})

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Function to run the algorithms

In [48]:
def RunLLM(llm, df, question_text, template, llmbode=0):
    inicio = time.time()
    result_list = []
    count = 0
    
    for text in df['product_name']:
        try: 
            question = f"{question_text} {text}"
            prompt = PromptTemplate.from_template(template)
            llm_chain = LLMChain(prompt=prompt, llm=llm)
            #print(f'{count} - Text: {text}')
            result = llm_chain.run(question)
            if (llmbode == 2):
                result_list.append(result.split('A lista é')[1])
            else:
                result_list.append(result)#.split('\n')[0])
            #print('-----------------------------------')
            count = count + 1
        except Exception as e:
            print(e)
            result_list.append('error')
            count = count + 1

    fim = time.time()

    tempo_decorrido = fim - inicio
    print("Tempo decorrido:", tempo_decorrido, "segundos\n")
    
    return result_list

# Running algorithms

## Phi-Bode

In [51]:
question_text = f"Baseado na seguinte frase, retorne uma lista das palavras mais relevantes:"           
template = """Questão: {question}
Resposta: A lista é """

all_results.append(RunLLM(llm_bode, df, question_text, template, llmbode=1))

Tempo decorrido: 0.1425788402557373 segundos

[['Questão: Baseado na seguinte frase, retorne uma lista das palavras mais relevantes: Copo Acrílico Com Canudo 500ml Rocie\nResposta: A lista é \n\n1. Copo Acrílico\n2. Com Canudo\n3. 500ml\n4. Rocie\n']]


## Vlt5

In [6]:
question_text = f"Based in the following phrase, return a list of the most relevant words:"           
template = """Question: {question}
Answer: The list is """

all_results.append(RunLLM(llm_vlt5, df, question_text, template))

/home/myuser/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.2.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/home/myuser/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/home/myuser/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


0 - Text: Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco
1 - Text: Copo Acrílico Com Canudo 500ml Rocie
2 - Text: Panela de Pressão Elétrica Philips Walita Daily 5L com Timer
3 - Text: Betoneira Columbus - Roma Brinquedos
4 - Text: Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com Conversor Digital 4 HDMI 2 USB WebOS 3.5 Painel Ips HDR e Magic Mobile Connection
5 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum
6 - Text: Ventilador de Teto Ventisol Fenix Premium Branco 3 velocidades com Controle Remoto
7 - Text: Kit 10 Peças Fita Led 12v 5050 300 Leds Rgb + Fonte
8 - Text: Aspirador de Pó Philco Rapid 1000N Vermelho/Preto - 1000W
9 - Text: Conjunto de Panelas Antiaderente Turim 10 Peças Vermelha Tramontina
10 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Dourado
11 - Text: Tênis Casual

## Google

In [7]:
all_results.append(RunLLM(llm_google, df, question_text, template))

0 - Text: Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco
1 - Text: Copo Acrílico Com Canudo 500ml Rocie
2 - Text: Panela de Pressão Elétrica Philips Walita Daily 5L com Timer
3 - Text: Betoneira Columbus - Roma Brinquedos
4 - Text: Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com Conversor Digital 4 HDMI 2 USB WebOS 3.5 Painel Ips HDR e Magic Mobile Connection
5 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum
6 - Text: Ventilador de Teto Ventisol Fenix Premium Branco 3 velocidades com Controle Remoto
7 - Text: Kit 10 Peças Fita Led 12v 5050 300 Leds Rgb + Fonte
8 - Text: Aspirador de Pó Philco Rapid 1000N Vermelho/Preto - 1000W
9 - Text: Conjunto de Panelas Antiaderente Turim 10 Peças Vermelha Tramontina
10 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Dourado
11 - Text: Tênis Casual

## BigScience

In [29]:
all_results.append(RunLLM(llm_bigscience, df, question_text, template))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
0 - Text: Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco
1 - Text: Copo Acrílico Com Canudo 500ml Rocie
2 - Text: Panela de Pressão Elétrica Philips Walita Daily 5L com Timer
3 - Text: Betoneira Columbus - Roma Brinquedos
4 - Text: Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com Conversor Digital 4 HDMI 2 USB WebOS 3.5 Painel Ips HDR e Magic Mobile Connection
5 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum
6 - Text: Ventilador de Teto Ventisol Fenix Premium Branco 3 velocidades com Controle Remoto
7 - Text: Kit 10 Peças Fita Led 12v 5050 300 Leds Rgb + Fonte
8 - Text: Aspirador de Pó Philco Rapid 1000N Vermelh

## Face

In [30]:
all_results.append(RunLLM(llm_face, df, question_text, template))

0 - Text: Notebook Asus Vivobook Max X541NA-GO472T Intel Celeron Quad Core 4GB 500GB Tela LED 15,6" Windows - 10 Branco
1 - Text: Copo Acrílico Com Canudo 500ml Rocie
2 - Text: Panela de Pressão Elétrica Philips Walita Daily 5L com Timer
3 - Text: Betoneira Columbus - Roma Brinquedos
4 - Text: Smart TV LED 43" LG 43UJ6525 Ultra HD 4K com Conversor Digital 4 HDMI 2 USB WebOS 3.5 Painel Ips HDR e Magic Mobile Connection
5 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Platinum
6 - Text: Ventilador de Teto Ventisol Fenix Premium Branco 3 velocidades com Controle Remoto
7 - Text: Kit 10 Peças Fita Led 12v 5050 300 Leds Rgb + Fonte
8 - Text: Aspirador de Pó Philco Rapid 1000N Vermelho/Preto - 1000W
9 - Text: Conjunto de Panelas Antiaderente Turim 10 Peças Vermelha Tramontina
10 - Text: Smartphone Motorola Moto G 5S Dual Chip Android 7.1.1 Nougat Tela 5.2" Snapdragon 430 32GB 4G Câmera 16MP - Dourado
11 - Text: Tênis Casual

## KeyBERT

In [25]:
import sys
sys.path.append('../terms_extraction/')

from terms_extractor import terms_extractor

from keybert import KeyBERT

inicio = time.time()

kw_model = KeyBERT()
keywords = kw_model.extract_keywords(' '.join(df['product_name'].tolist()), top_n=250)

keywords = [item[0] for item in keywords]

_, processed_df_keybert = terms_extractor(data=df, extract=keywords, 
                                          target_column='product_name', verbose=1)

fim = time.time()
tempo_decorrido = fim - inicio
print("Tempo decorrido:", tempo_decorrido, "segundos")

=============================== RUNNING THE PREPROCESSING ===============================

Defined pipeline: ['transform_to_lowercase', 'remove_special_characters'] 

Preprocess --> transform_to_lowercase
Converting text to lowercase...
Done!

Preprocess --> remove_special_characters
Removing special characters from text...
Done!

=============================== END OF THE PREPROCESSING ================================

============================== RUNNING THE TERMS EXTRACTION =============================

This stage can take several minutes...

Terms extraction with your list...


Processing column prep: 100%|██████████| 100/100 [00:00<00:00, 40006.71it/s]

Done!

============================== END OF THE TERMS EXTRACTION ==============================

Tempo decorrido: 1.8102798461914062 segundos


## USKE

In [31]:
import sys
sys.path.append('../terms_extraction/')

from terms_extractor import terms_extractor

inicio = time.time()

extract, processed_df = terms_extractor(data=df, measures=['yake', 'tfidf', 'pmi', 'mle', 'dice', 'count'], 
                                        target_column='product_name', number_to_extract=100, 
                                        stopwords=True, language='portuguese', verbose=1)

fim = time.time()

def merge_list(row):
    merged_list = []
    for sublist in row:
        merged_list.extend(sublist)
    merged_list = list(set(merged_list))  # Removendo itens repetidos
    return merged_list

# Aplicando a função para cada linha e criando uma nova coluna com o resultado
processed_df['merged'] = processed_df[['yake', 'tfidf', 'pmi', 'mle', 'dice', 'count']].apply(merge_list, axis=1)
processed_df.reset_index(drop=True, inplace=True)

display(processed_df)

tempo_decorrido = fim - inicio
print("Tempo decorrido:", tempo_decorrido, "segundos")

=============================== RUNNING THE PREPROCESSING ===============================

Defined pipeline: ['transform_to_lowercase', 'remove_special_characters', 'remove_stopwords'] 

Preprocess --> transform_to_lowercase
Converting text to lowercase...
Done!

Preprocess --> remove_special_characters
Removing special characters from text...
Done!

Preprocess --> remove_stopwords
Getting stopword list and removing from text...
Done!

=============================== END OF THE PREPROCESSING ================================

============================ RUNNING THE TERMS IDENTIFICATION ===========================

Performing the terms identification with ['yake', 'tfidf', 'pmi', 'mle', 'dice', 'count'] measures...

You're gonna identify 100 terms per measure!

Applying yake measure...


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Applying tfidf measure...
Applying pmi measure...
Applying mle measure...
Applying dice measure...
Applying count measure...

Done!

============================ END OF THE TERMS IDENTIFICATION ============================

============================== RUNNING THE TERMS EXTRACTION =============================

This stage can take several minutes...

Terms extraction with yake...


Processing column prep: 100%|██████████| 100/100 [00:00<00:00, 89736.93it/s]


Terms extraction with tfidf...


...

Terms extraction with count...


Processing column prep: 100%|██████████| 100/100 [00:00<00:00, 68680.27it/s]

Done!

============================== END OF THE TERMS EXTRACTION ==============================



,product_name,prep,yake,tfidf,pmi,mle,dice,count,merged
0,Notebook Asus Vivobook Max X541NA-GO472T Intel...,notebook asus vivobook max x541na go472t intel...,"[notebook, asus, quad, core, tela, led, window...","[notebook, asus, x541na, led, 10, branco]",[notebook],"[asus, quad, core, tela, led, 6, 10, branco]",[notebook],"[asus, quad, core, tela, led, 6, 10, branco]","[6, windows, branco, quad, asus, notebook, x54..."
1,Copo Acrílico Com Canudo 500ml Rocie,copo acrilico canudo 500ml rocie,[copo],[],[],[],[],[],[copo]
2,Panela de Pressão Elétrica Philips Walita Dail...,panela pressao eletrica philips walita daily 5...,"[eletrica, philips]","[eletrica, philips, walita]",[],[eletrica],[],[eletrica],"[philips, walita, eletrica]"
3,Betoneira Columbus - Roma Brinquedos,betoneira columbus roma brinquedos,[brinquedos],[brinquedos],[],[],[],[],[brinquedos]
4,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",smart tv led 43 lg 43uj6525 ultra hd 4k conver...,"[smart, led, ultra, conversor, digital, hdmi, ...","[smart, tv, led, 43, lg, ultra, hd, 4k, usb]",[],"[smart, tv, led, ultra, hd, 4k, conversor, dig...",[],"[smart, tv, led, ultra, hd, 4k, conversor, dig...","[ultra, conversor, 4k, 4, usb, magic, tv, 43, ..."
...,...,...,...,...,...,...,...,...,...
95,Robô Aspirador de Pó Ropo Glass Limpeza Progra...,robo aspirador po ropo glass limpeza programav...,"[aspirador, limpeza, controle, remoto, preto]","[aspirador, po, limpeza, controle, preto]",[],"[limpeza, controle, preto]",[],"[limpeza, controle, preto]","[limpeza, preto, controle, aspirador, po, remoto]"
96,Modelador de Cachos - Preto,modelador cachos preto,[preto],[preto],[],[preto],[],[preto],[preto]
97,Protetor De Sofá Manu Para 2 E 3 Lugares Dupla...,protetor sofa manu 2 3 lugares dupla face teci...,[],[],[],"[2, 3]",[],"[2, 3]","[2, 3]"
98,Capa De Banco Para Moto Termica Impermeavel Ve...,capa banco moto termica impermeavel ventilada ...,"[capa, moto, preta]","[capa, moto, preta]",[],"[capa, moto, preta]",[],"[capa, moto, preta]","[capa, moto, preta]"


Tempo decorrido: 0.31035375595092773 segundos


# Transforming the results into CSV

In [32]:
df_final = pd.DataFrame(all_results).transpose()
df_final.columns = ['Bode', 'vlt5', 'Google', 'BigScience', 'Facebook']
df_final = processed_df.merge(df_final, left_index=True, right_index=True)
df_final['keybert'] = processed_df_keybert['your list']

display(df_final)

df_final.to_csv('Results_b2w.csv', index=False)

,product_name,prep,yake,tfidf,pmi,mle,dice,count,merged,vlt5,Google,BigScience,Facebook
0,Notebook Asus Vivobook Max X541NA-GO472T Intel...,notebook asus vivobook max x541na go472t intel...,"[notebook, asus, quad, core, tela, led, window...","[notebook, asus, x541na, led, 10, branco]",[notebook],"[asus, quad, core, tela, led, 6, 10, branco]",[notebook],"[asus, quad, core, tela, led, 6, 10, branco]","[6, windows, branco, quad, asus, notebook, x54...","Asus Vivobook Max, Windows 10 Branco, Windows ...","asus, core, gb, tela, vivobook, window",Asus Vivobook Max X541NA-GO472T Intel Celeron...,"I'm not sure what you mean by ""The list"" but ..."
1,Copo Acrílico Com Canudo 500ml Rocie,copo acrilico canudo 500ml rocie,[copo],[],[],[],[],[],[copo],"Copo Acrílico, Copo Acrílico, Copo Acríli","acilico, canudo, com, rocie","Rocie, 500ml, Com, Canudo, Acrílico, Copo, Ro...","Do you know that the word ""copo"" means ""littl..."
2,Panela de Pressão Elétrica Philips Walita Dail...,panela pressao eletrica philips walita daily 5...,"[eletrica, philips]","[eletrica, philips, walita]",[],[eletrica],[],[eletrica],"[philips, walita, eletrica]",Panela de Pressão Elétrica Philips Walita Dail...,"presso, panela, philips, timer","['Panela de Pressão Elétrica', 'Philips Walit...","I'm not sure what you are trying to say, but ..."
3,Betoneira Columbus - Roma Brinquedos,betoneira columbus roma brinquedos,[brinquedos],[brinquedos],[],[],[],[],[brinquedos],"Betoneira Columbus, Roma Brinquedos, Roma Brin...","brinquedos, columbus, roma","['Betoneira','Columbus','Roma','Brinquedos']","I'm not sure what you are trying to say, but ..."
4,"Smart TV LED 43"" LG 43UJ6525 Ultra HD 4K com C...",smart tv led 43 lg 43uj6525 ultra hd 4k conver...,"[smart, led, ultra, conversor, digital, hdmi, ...","[smart, tv, led, 43, lg, ultra, hd, 4k, usb]",[],"[smart, tv, led, ultra, hd, 4k, conversor, dig...",[],"[smart, tv, led, ultra, hd, 4k, conversor, dig...","[ultra, conversor, 4k, 4, usb, magic, tv, 43, ...","LG 43UJ6525, LG 43UJ6525, LG 43","connection, hdmi, ips, tv",,I'm not sure what you are trying to say. Wha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Robô Aspirador de Pó Ropo Glass Limpeza Progra...,robo aspirador po ropo glass limpeza programav...,"[aspirador, limpeza, controle, remoto, preto]","[aspirador, po, limpeza, controle, preto]",[],"[limpeza, controle, preto]",[],"[limpeza, controle, preto]","[limpeza, preto, controle, aspirador, po, remoto]","application, phrase, remoto preto","aspirador, controle, remoto, ropo",,Robotics is the science and technology of des...
96,Modelador de Cachos - Preto,modelador cachos preto,[preto],[preto],[],[preto],[],[preto],[preto],"Modelador de Cachos, Modelador de Cachos, Mode...","cacho, modelador, preto","['black', 'curls', 'hair']",Pretos are a brand of cheese-flavored cornmea...
97,Protetor De Sofá Manu Para 2 E 3 Lugares Dupla...,protetor sofa manu 2 3 lugares dupla face teci...,[],[],[],"[2, 3]",[],"[2, 3]","[2, 3]","Caqui/Marrom, Protetor De Sofá Manu Para 2 E 3","face, lugares, protector","[protetor, sofá, manu, lugar, dupla, face, te...","I'm not sure what you mean by ""The list"" but ..."
98,Capa De Banco Para Moto Termica Impermeavel Ve...,capa banco moto termica impermeavel ventilada ...,"[capa, moto, preta]","[capa, moto, preta]",[],"[capa, moto, preta]",[],"[capa, moto, preta]","[capa, moto, preta]","Capa De Banco Para Motocicleta Cor Preta, Capa...","capa, moto, termica",""" black, ventilated, motorcycle, thermal, imp...","Do you know the meaning of the word ""Capa"" wh..."
